In [1]:
import os

OUTPUT_DIR = 'output_ruenrosberta_v5'
if not os.path.exists(OUTPUT_DIR):
    os.makedirs(OUTPUT_DIR)

In [2]:
class CFG:
    apex = True
    print_freq = 100
    num_workers = 8
    model = "ai-forever/ru-en-RoSBERTa"
    gradient_checkpointing = False
    scheduler = 'cosine'  # ['linear', 'cosine']
    batch_scheduler = True
    num_cycles = 0.5
    num_warmup_steps = 0
    epochs = 10
    encoder_lr = 2e-5
    decoder_lr = 2e-5
    min_lr = 1e-6
    eps = 1e-6
    betas = (0.9, 0.999)
    batch_size = 16
    max_len = 512
    weight_decay = 0.01
    gradient_accumulation_steps = 1
    max_grad_norm = 1000
    # target_cols = ['Классификатор 1 уровня']
    seed = 42
    n_fold = 6
    trn_fold = [0]
    train = True

In [3]:
from transformers import get_linear_schedule_with_warmup, get_cosine_schedule_with_warmup
from transformers import AutoTokenizer, AutoModel, AutoConfig
from sklearn.metrics import f1_score, accuracy_score
from torch.utils.data import DataLoader, Dataset
from torch.optim import AdamW
import torch.nn as nn
import torch
import torch.nn.functional as F
from sklearn.model_selection import StratifiedKFold
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold
from sklearn.preprocessing import LabelEncoder
from tqdm.auto import tqdm
import pickle
import pandas as pd
import numpy as np
import gc
import re
import time
import math
import random
import warnings
warnings.filterwarnings("ignore")

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
os.environ["TOKENIZERS_PARALLELISM"] = "false"

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [4]:
def preprocess(df):
    df["Классификатор 1 уровня"] = df["Классификатор 1 уровня"].apply(
        lambda x: x.strip())
    df["Классификатор 2 уровня"] = df["Классификатор 2 уровня"].apply(
        lambda x: x.strip())
    return df


def get_score(y_trues, class1_predictions, class2_predictions):
    class1_predictions = [np.argmax(el) for el in class1_predictions]
    class2_predictions = [np.argmax(el) for el in class2_predictions]

    class1_score = accuracy_score(y_trues[:, 0], class1_predictions)
    class2_score = accuracy_score(y_trues[:, 1], class2_predictions)
    return class1_score, class2_score


def get_logger(filename=os.path.join(OUTPUT_DIR, 'train')):
    from logging import getLogger, INFO, StreamHandler, FileHandler, Formatter
    logger = getLogger(__name__)
    logger.setLevel(INFO)
    handler1 = StreamHandler()
    handler1.setFormatter(Formatter("%(message)s"))
    handler2 = FileHandler(filename=f"{filename}.log")
    handler2.setFormatter(Formatter("%(message)s"))
    logger.addHandler(handler1)
    logger.addHandler(handler2)
    return logger


LOGGER = get_logger()


def seed_everything(seed=42):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True


seed_everything(seed=42)

In [5]:
train = pd.read_csv("../case/Aug_data (1).csv",
                    sep="|").rename(columns={"q": "Вопрос пользователя"})
train = preprocess(train)
train.head()

,Unnamed: 0,Вопрос пользователя,Ответ сотрудника,Вопрос из БЗ,Ответ из БЗ,Классификатор 1 уровня,Классификатор 2 уровня,aug_questions,Paraphrase 1,Paraphrase 2,Paraphrase 3
0,0,Здравствуйте! Можно уточнить причины Правилhtt...,Добрый день!\nЧто нельзя публиковать на RUTUBE...,Что нельзя публиковать на RUTUBE?,Чужой контент без разрешения автора или правоо...,МОДЕРАЦИЯ,Отклонение/блокировка видео,Здравствуйте! Можно уточнить причины Правилhtt...,"Добрый день! Подскажите, по каким причинам ваш...","Здравствуйте! Не могли бы вы разъяснить, за чт...","Приветствую, хочу уточнить, на каком основании..."
1,1,"Добрый вечер, какой топ причин блокировки виде...",Добрый вечер!\nЧто заперщено публиковать на RU...,Что нельзя публиковать на RUTUBE?,Чужой контент без разрешения автора или правоо...,МОДЕРАЦИЯ,Отклонение/блокировка видео,"Добрый вечер, какой топ основ блокировки видео...","1. Здравствуйте, подскажите, пожалуйста, топ п...","2. Привет, можно узнать, какие основные причин...","3. Добрый день, какие самые частые причины уда..."
2,2,"Все пишут, что монетизация на рутубе отключает...","Добрый день! \nМонетизация может отключиться, ...",Почему могут отключить монетизацию из-за автор...,"Монетизация может отключиться, если на вашем к...",МОНЕТИЗАЦИЯ,Отключение/подключение монетизации,"Все изображают, что монетизация на рутубе откл...","1. Читаю, что на Рутубе монетизацию само по се...","2. Все говорят, что на Рутубе монетизацию само...","3. Многие пишут, что на Рутубе монетизация сам..."
3,3,Что запрещено в монетизации и что можно выклад...,"Здравствуйте!\nМонетизация может отключиться, ...",Почему могут отключить монетизацию из-за автор...,"Монетизация может отключиться, если на вашем к...",МОНЕТИЗАЦИЯ,Отключение/подключение монетизации,Что запрещено в монетизации и что можно мостить?,"1. ""Какие ограничения на монетизацию и что раз...","2. ""Что нельзя монетизировать и что можно публ...","3. ""Какие материалы запрещено монетизировать и..."
4,4,"Чтобы не отключали монетизацию, надо, чтобы я ...","Для монетизации можно использовать то, что вы ...",Почему могут отключить монетизацию из-за автор...,"Монетизация может отключиться, если на вашем к...",МОНЕТИЗАЦИЯ,Отключение/подключение монетизации,"Чтобы не отключали монетизацию, надо, чтобы я ...","1. Так, чтобы мне не вырубили монету, это мне ...",NaN,"2. Если не хочу, чтобы меня отключили от монет..."


In [6]:
class1_le = LabelEncoder()
class2_le = LabelEncoder()
class1_le.fit(train["Классификатор 1 уровня"].tolist())
class2_le.fit(train["Классификатор 2 уровня"].tolist())
train["Классификатор 1 уровня"] = class1_le.transform(
    train["Классификатор 1 уровня"].tolist())
train["Классификатор 2 уровня"] = class2_le.transform(
    train["Классификатор 2 уровня"].tolist())

with open(os.path.join(OUTPUT_DIR, "class1_le.pkl"), "wb") as f:
    pickle.dump(class1_le, f)
with open(os.path.join(OUTPUT_DIR, "class2_le.pkl"), "wb") as f:
    pickle.dump(class2_le, f)

In [7]:
Fold = MultilabelStratifiedKFold(n_splits=CFG.n_fold,
                                 shuffle=True, random_state=CFG.seed)
for n, (train_index, val_index) in enumerate(Fold.split(train, train[["Классификатор 1 уровня", "Классификатор 2 уровня"]].values)):
    train.loc[val_index, 'fold'] = int(n)
train['fold'] = train['fold'].astype(int)
train.head()

,Unnamed: 0,Вопрос пользователя,Ответ сотрудника,Вопрос из БЗ,Ответ из БЗ,Классификатор 1 уровня,Классификатор 2 уровня,aug_questions,Paraphrase 1,Paraphrase 2,Paraphrase 3,fold
0,0,Здравствуйте! Можно уточнить причины Правилhtt...,Добрый день!\nЧто нельзя публиковать на RUTUBE...,Что нельзя публиковать на RUTUBE?,Чужой контент без разрешения автора или правоо...,3,14,Здравствуйте! Можно уточнить причины Правилhtt...,"Добрый день! Подскажите, по каким причинам ваш...","Здравствуйте! Не могли бы вы разъяснить, за чт...","Приветствую, хочу уточнить, на каком основании...",3
1,1,"Добрый вечер, какой топ причин блокировки виде...",Добрый вечер!\nЧто заперщено публиковать на RU...,Что нельзя публиковать на RUTUBE?,Чужой контент без разрешения автора или правоо...,3,14,"Добрый вечер, какой топ основ блокировки видео...","1. Здравствуйте, подскажите, пожалуйста, топ п...","2. Привет, можно узнать, какие основные причин...","3. Добрый день, какие самые частые причины уда...",4
2,2,"Все пишут, что монетизация на рутубе отключает...","Добрый день! \nМонетизация может отключиться, ...",Почему могут отключить монетизацию из-за автор...,"Монетизация может отключиться, если на вашем к...",4,15,"Все изображают, что монетизация на рутубе откл...","1. Читаю, что на Рутубе монетизацию само по се...","2. Все говорят, что на Рутубе монетизацию само...","3. Многие пишут, что на Рутубе монетизация сам...",0
3,3,Что запрещено в монетизации и что можно выклад...,"Здравствуйте!\nМонетизация может отключиться, ...",Почему могут отключить монетизацию из-за автор...,"Монетизация может отключиться, если на вашем к...",4,15,Что запрещено в монетизации и что можно мостить?,"1. ""Какие ограничения на монетизацию и что раз...","2. ""Что нельзя монетизировать и что можно публ...","3. ""Какие материалы запрещено монетизировать и...",1
4,4,"Чтобы не отключали монетизацию, надо, чтобы я ...","Для монетизации можно использовать то, что вы ...",Почему могут отключить монетизацию из-за автор...,"Монетизация может отключиться, если на вашем к...",4,15,"Чтобы не отключали монетизацию, надо, чтобы я ...","1. Так, чтобы мне не вырубили монету, это мне ...",NaN,"2. Если не хочу, чтобы меня отключили от монет...",1


In [8]:
extra = train[["Вопрос пользователя", "Классификатор 1 уровня",
               "Классификатор 2 уровня", "fold"]]
extra1 = train[["aug_questions", "Классификатор 1 уровня", "Классификатор 2 уровня",
                "fold"]].rename(columns={"aug_questions": "Вопрос пользователя"})
extra1 = extra1[extra1.fold != 0]
extra2 = train[["Paraphrase 1", "Классификатор 1 уровня", "Классификатор 2 уровня",
                "fold"]].rename(columns={"Paraphrase 1": "Вопрос пользователя"})
extra2 = extra2[extra2.fold != 0]
extra3 = train[["Paraphrase 2", "Классификатор 1 уровня", "Классификатор 2 уровня",
                "fold"]].rename(columns={"Paraphrase 2": "Вопрос пользователя"})
extra3 = extra3[extra3.fold != 0]
extra4 = train[["Paraphrase 3", "Классификатор 1 уровня", "Классификатор 2 уровня",
                "fold"]].rename(columns={"Paraphrase 3": "Вопрос пользователя"})
extra4 = extra4[extra4.fold != 0]
extra5 = train[["Вопрос из БЗ", "Классификатор 1 уровня", "Классификатор 2 уровня",
                "fold"]].rename(columns={"Вопрос из БЗ": "Вопрос пользователя"})
extra5 = extra5[extra5.fold != 0]

train = pd.concat([extra, extra1, extra2, extra3, extra4, extra5], axis=0)

print(train.shape)
train.drop_duplicates(inplace=True)
train.dropna(inplace=True)
train.reset_index(inplace=True, drop=True)
print(train.shape)
train.head()

(4141, 4)
(3827, 4)


,Вопрос пользователя,Классификатор 1 уровня,Классификатор 2 уровня,fold
0,Здравствуйте! Можно уточнить причины Правилhtt...,3,14,3
1,"Добрый вечер, какой топ причин блокировки виде...",3,14,4
2,"Все пишут, что монетизация на рутубе отключает...",4,15,0
3,Что запрещено в монетизации и что можно выклад...,4,15,1
4,"Чтобы не отключали монетизацию, надо, чтобы я ...",4,15,1


In [9]:
tokenizer = AutoTokenizer.from_pretrained(CFG.model)
tokenizer.save_pretrained(os.path.join(OUTPUT_DIR, 'tokenizer'))
CFG.tokenizer = tokenizer

lengths = []
tk0 = tqdm(train['Вопрос пользователя'].fillna("").values, total=len(train))
for text in tk0:
    length = len(tokenizer(text, add_special_tokens=False)['input_ids'])
    lengths.append(length)
CFG.max_len = max(lengths) + 2  # cls & sep
LOGGER.info(f"max_len: {CFG.max_len}")
CFG.max_len = 512

  0%|          | 0/3827 [00:00<?, ?it/s]

max_len: 332


In [10]:
def prepare_input(cfg, text):
    inputs = cfg.tokenizer.encode_plus(
        text,
        return_tensors=None,
        add_special_tokens=True,
        max_length=CFG.max_len,
        pad_to_max_length=True,
        truncation=True
    )
    for k, v in inputs.items():
        inputs[k] = torch.tensor(v, dtype=torch.long)
    return inputs


class TrainDataset(Dataset):
    def __init__(self, cfg, df):
        self.cfg = cfg
        self.texts = df['Вопрос пользователя'].values
        self.class1 = df["Классификатор 1 уровня"].values
        self.class2 = df["Классификатор 2 уровня"].values

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, item):
        inputs = prepare_input(self.cfg, self.texts[item])
        class1 = torch.tensor(self.class1[item], dtype=torch.long)
        class2 = torch.tensor(self.class2[item], dtype=torch.long)
        return inputs, class1, class2


def collate(inputs):
    mask_len = int(inputs["attention_mask"].sum(axis=1).max())
    for k, v in inputs.items():
        inputs[k] = inputs[k][:, :mask_len]
    return inputs

In [11]:
def average_pool(last_hidden_states,
                 attention_mask):
    last_hidden = last_hidden_states.masked_fill(
        ~attention_mask[..., None].bool(), 0.0)
    return last_hidden.sum(dim=1) / attention_mask.sum(dim=1)[..., None]


def pool(hidden_state, mask, pooling_method="mean"):
    if pooling_method == "mean":
        s = torch.sum(hidden_state * mask.unsqueeze(-1).float(), dim=1)
        d = mask.sum(axis=1, keepdim=True).float()
        return s / d
    elif pooling_method == "cls":
        return hidden_state[:, 0]


def mean_pooling(model_output, attention_mask):
    # First element of model_output contains all token embeddings
    token_embeddings = model_output[0]
    input_mask_expanded = attention_mask.unsqueeze(
        -1).expand(token_embeddings.size()).float()
    sum_embeddings = torch.sum(token_embeddings * input_mask_expanded, 1)
    sum_mask = torch.clamp(input_mask_expanded.sum(1), min=1e-9)
    return sum_embeddings / sum_mask


class CustomModel(nn.Module):
    def __init__(self, cfg, config_path=None, pretrained=False):
        super().__init__()
        self.cfg = cfg
        if config_path is None:
            self.config = AutoConfig.from_pretrained(
                cfg.model, output_hidden_states=True)
            self.config.hidden_dropout = 0.
            self.config.hidden_dropout_prob = 0.
            self.config.attention_dropout = 0.
            self.config.attention_probs_dropout_prob = 0.
            LOGGER.info(self.config)
        else:
            self.config = torch.load(config_path)
        if pretrained:
            self.model = AutoModel.from_pretrained(
                cfg.model, config=self.config)
        else:
            self.model = AutoModel(self.config)
        if self.cfg.gradient_checkpointing:
            self.model.gradient_checkpointing_enable()

        self.fc_class1 = nn.Linear(self.config.hidden_size, 11)
        self.fc_class2 = nn.Linear(self.config.hidden_size, 39)
        self._init_weights(self.fc_class1)
        self._init_weights(self.fc_class2)
        # self.freeze_model()

    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            module.weight.data.normal_(
                mean=0.0, std=self.config.initializer_range)
            if module.bias is not None:
                module.bias.data.zero_()
        elif isinstance(module, nn.Embedding):
            module.weight.data.normal_(
                mean=0.0, std=self.config.initializer_range)
            if module.padding_idx is not None:
                module.weight.data[module.padding_idx].zero_()
        elif isinstance(module, nn.LayerNorm):
            module.bias.data.zero_()
            module.weight.data.fill_(1.0)

    def freeze_model(self):
        for name, param in self.model.named_parameters():
            param.requires_grad = False

    def get_vector_e5(self, inputs):
        outputs = self.model(**inputs)
        feature = average_pool(outputs.last_hidden_state,
                               inputs['attention_mask'])
        return F.normalize(feature, p=2, dim=1)

    def get_vector_sbert(self, inputs):
        outputs = self.model(**inputs)
        sentence_embeddings = mean_pooling(outputs, inputs['attention_mask'])
        return sentence_embeddings

    def get_vector_rubert(self, inputs):
        outputs = self.model(**inputs)
        embeddings = outputs.last_hidden_state[:, 0, :]
        embeddings = torch.nn.functional.normalize(embeddings)
        return embeddings

    def get_vector_ruenrosberta(self, inputs):
        outputs = self.model(**inputs)
        embeddings = pool(
            outputs.last_hidden_state,
            inputs["attention_mask"],
            pooling_method="cls"  # or try "mean"
        )
        return F.normalize(embeddings, p=2, dim=1)

    def get_vector_labse(self, inputs):
        outputs = self.model(**inputs)
        feature = outputs.pooler_output
        return torch.nn.functional.normalize(feature)

    def head1(self, feature):
        output_class1 = self.fc_class1(feature)
        return output_class1

    def head2(self, feature):
        output_class2 = self.fc_class2(feature)
        return output_class2

    def inference(self, inputs):
        feature = self.get_vector_ruenrosberta(inputs)
        output_class1 = self.fc_class1(feature)
        output_class2 = self.fc_class2(feature)

        return output_class1, output_class2

In [12]:
class AverageMeter(object):
    """Computes and stores the average and current value"""

    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count


def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)


def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (remain %s)' % (asMinutes(s), asMinutes(rs))

In [13]:
def train_fn(fold, train_loader, model, criterion, optimizer, epoch, scheduler, device):
    model.train()
    scaler = torch.cuda.amp.GradScaler(enabled=CFG.apex)
    losses = AverageMeter()
    start = end = time.time()
    global_step = 0
    for step, (inputs, class1, class2) in enumerate(train_loader):
        inputs = collate(inputs)
        for k, v in inputs.items():
            inputs[k] = v.to(device)
        class1 = class1.to(device)
        class2 = class2.to(device)

        batch_size = class1.size(0)
        with torch.cuda.amp.autocast(enabled=CFG.apex):
            feature = model.get_vector_ruenrosberta(inputs)
            pred1 = model.head1(feature)
            pred2 = model.head2(feature)
            loss = criterion(pred1, class1) + criterion(pred2, class2)
        if CFG.gradient_accumulation_steps > 1:
            loss = loss / CFG.gradient_accumulation_steps
        losses.update(loss.item(), batch_size)
        scaler.scale(loss).backward()
        grad_norm = torch.nn.utils.clip_grad_norm_(
            model.parameters(), CFG.max_grad_norm)
        if (step + 1) % CFG.gradient_accumulation_steps == 0:
            scaler.step(optimizer)
            scaler.update()
            optimizer.zero_grad()
            global_step += 1
            if CFG.batch_scheduler:
                scheduler.step()
        end = time.time()
        if step % CFG.print_freq == 0 or step == (len(train_loader)-1):
            print('Epoch: [{0}][{1}/{2}] '
                  'Elapsed {remain:s} '
                  'Loss: {loss.val:.4f}({loss.avg:.4f}) '
                  'Grad: {grad_norm:.4f}  '
                  'LR: {lr:.8f}  '
                  .format(epoch+1, step, len(train_loader),
                          remain=timeSince(start, float(
                              step+1)/len(train_loader)),
                          loss=losses,
                          grad_norm=grad_norm,
                          lr=scheduler.get_lr()[0]))
    return losses.avg


def valid_fn(valid_loader, model, criterion, device):
    losses = AverageMeter()
    model.eval()
    preds1 = []
    preds2 = []
    start = end = time.time()
    for step, (inputs, label1, label2) in enumerate(valid_loader):
        inputs = collate(inputs)
        for k, v in inputs.items():
            inputs[k] = v.to(device)
        label1 = label1.to(device)
        label2 = label2.to(device)

        batch_size = label1.size(0)
        with torch.no_grad():
            pred1, pred2 = model.inference(inputs)
            loss = criterion(pred1, label1) + criterion(pred2, label2)
        if CFG.gradient_accumulation_steps > 1:
            loss = loss / CFG.gradient_accumulation_steps
        losses.update(loss.item(), batch_size)
        preds1.append(pred1.to('cpu').numpy())
        preds2.append(pred2.to('cpu').numpy())
        end = time.time()
        if step % CFG.print_freq == 0 or step == (len(valid_loader)-1):
            print('EVAL: [{0}/{1}] '
                  'Elapsed {remain:s} '
                  'Loss: {loss.val:.4f}({loss.avg:.4f}) '
                  .format(step, len(valid_loader),
                          loss=losses,
                          remain=timeSince(start, float(step+1)/len(valid_loader))))
    predictions1 = np.concatenate(preds1)
    predictions2 = np.concatenate(preds2)
    return losses.avg, predictions1, predictions2

In [14]:
def train_loop(folds, fold):
    LOGGER.info(f"========== fold: {fold} training ==========")
    # ====================================================
    # loader
    # ====================================================
    train_folds = folds[folds['fold'] != fold].reset_index(drop=True)

    valid_folds = folds[folds['fold'] == fold].reset_index(drop=True)
    valid_labels = valid_folds[[
        "Классификатор 1 уровня", "Классификатор 2 уровня"]].values

    train_dataset = TrainDataset(CFG, train_folds)
    valid_dataset = TrainDataset(CFG, valid_folds)

    train_loader = DataLoader(train_dataset,
                              batch_size=CFG.batch_size,
                              shuffle=True,
                              num_workers=CFG.num_workers, pin_memory=True, drop_last=True)
    valid_loader = DataLoader(valid_dataset,
                              batch_size=CFG.batch_size * 2,
                              shuffle=False,
                              num_workers=CFG.num_workers, pin_memory=True, drop_last=False)

    # ====================================================
    # model & optimizer
    # ====================================================
    model = CustomModel(CFG, config_path=None, pretrained=True)
    torch.save(model.config, os.path.join(OUTPUT_DIR, 'config.pth'))
    model.to(device)

    def get_optimizer_params(model, encoder_lr, decoder_lr, weight_decay=0.0):
        param_optimizer = list(model.named_parameters())
        no_decay = ["bias", "LayerNorm.bias", "LayerNorm.weight"]
        optimizer_parameters = [
            {'params': [p for n, p in model.model.named_parameters() if not any(nd in n for nd in no_decay)],
             'lr': encoder_lr, 'weight_decay': weight_decay},
            {'params': [p for n, p in model.model.named_parameters() if any(nd in n for nd in no_decay)],
             'lr': encoder_lr, 'weight_decay': 0.0},
            {'params': [p for n, p in model.named_parameters() if "model" not in n],
             'lr': decoder_lr, 'weight_decay': 0.0}
        ]
        return optimizer_parameters

    optimizer_parameters = get_optimizer_params(model,
                                                encoder_lr=CFG.encoder_lr,
                                                decoder_lr=CFG.decoder_lr,
                                                weight_decay=CFG.weight_decay)
    optimizer = AdamW(optimizer_parameters, lr=CFG.encoder_lr,
                      eps=CFG.eps, betas=CFG.betas)

    # ====================================================
    # scheduler
    # ====================================================
    def get_scheduler(cfg, optimizer, num_train_steps):
        if cfg.scheduler == 'linear':
            scheduler = get_linear_schedule_with_warmup(
                optimizer, num_warmup_steps=cfg.num_warmup_steps, num_training_steps=num_train_steps
            )
        elif cfg.scheduler == 'cosine':
            scheduler = get_cosine_schedule_with_warmup(
                optimizer, num_warmup_steps=cfg.num_warmup_steps, num_training_steps=num_train_steps, num_cycles=cfg.num_cycles
            )
        return scheduler

    num_train_steps = int(len(train_folds) / CFG.batch_size * CFG.epochs)
    scheduler = get_scheduler(CFG, optimizer, num_train_steps)

    # ====================================================
    # loop
    # ====================================================
    criterion = nn.CrossEntropyLoss()  # МБ добавить веса в лосс
    best_score = -1 * float('inf')
    for epoch in range(CFG.epochs):
        start_time = time.time()
        # train
        avg_loss = train_fn(fold, train_loader, model,
                            criterion, optimizer, epoch, scheduler, device)
        # eval
        avg_val_loss, predictions1, predictions2 = valid_fn(
            valid_loader, model, criterion, device)

        # scoring
        score1, score2 = get_score(valid_labels, predictions1, predictions2)

        elapsed = time.time() - start_time

        LOGGER.info(
            f'Epoch {epoch+1} - avg_train_loss: {avg_loss:.4f}  avg_val_loss: {avg_val_loss:.4f}  time: {elapsed:.0f}s')
        LOGGER.info(f'Epoch {epoch+1} - Score: {score1:.4f} {score2:.4f}')
        score = score1 + score2

        if best_score < score:
            best_score = score
            LOGGER.info(
                f'Epoch {epoch+1} - Save Best Score: {best_score:.4f} Model')
            torch.save({'model': model.state_dict(),
                        'predictions1': predictions1, 'predictions2': predictions2},
                       os.path.join(OUTPUT_DIR, f"{CFG.model.replace('/', '-')}_fold{fold}_best.pth"))

    predictions1 = torch.load(os.path.join(OUTPUT_DIR, f"{CFG.model.replace('/', '-')}_fold{fold}_best.pth"),
                              map_location=torch.device('cpu'))['predictions1']
    predictions2 = torch.load(os.path.join(OUTPUT_DIR, f"{CFG.model.replace('/', '-')}_fold{fold}_best.pth"),
                              map_location=torch.device('cpu'))['predictions2']

    valid_folds["pred1"] = [np.argmax(el) for el in predictions1]
    valid_folds["pred2"] = [np.argmax(el) for el in predictions2]

    torch.cuda.empty_cache()
    gc.collect()

    return valid_folds

In [15]:
if __name__ == '__main__':
    def get_result(oof_df):
        label1 = oof_df["Классификатор 1 уровня"].tolist()
        label2 = oof_df["Классификатор 2 уровня"].tolist()
        predictions1 = oof_df["pred1"].tolist()
        predictions2 = oof_df["pred2"].tolist()
        score1 = accuracy_score(label1, predictions1)
        score2 = accuracy_score(label2, predictions2)
        LOGGER.info(f'Score: {score1:.4f} | {score2:.4f}')

    if CFG.train:
        oof_df = pd.DataFrame()
        for fold in range(CFG.n_fold):
            if fold in CFG.trn_fold:
                _oof_df = train_loop(train, fold)
                oof_df = pd.concat([oof_df, _oof_df])
                LOGGER.info(f"========== fold: {fold} result ==========")
                get_result(_oof_df)
        oof_df = oof_df.reset_index(drop=True)
        LOGGER.info(f"========== CV ==========")
        get_result(oof_df)
        oof_df.to_pickle(os.path.join(OUTPUT_DIR, 'oof_df.pkl'))

========== fold: 0 training ==========
RobertaConfig {
  "_name_or_path": "ai-forever/ru-en-RoSBERTa",
  "architectures": [
    "RobertaModel"
  ],
  "attention_dropout": 0.0,
  "attention_probs_dropout_prob": 0.0,
  "bos_token_id": 1,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout": 0.0,
  "hidden_dropout_prob": 0.0,
  "hidden_size": 1024,
  "initializer_range": 0.02,
  "intermediate_size": 4096,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 16,
  "num_hidden_layers": 24,
  "output_hidden_states": true,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "torch_dtype": "float32",
  "transformers_version": "4.45.1",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_size": 98505
}

Some weights of RobertaModel were not initialized from the model checkpoint at ai-forever/ru-en-RoSBERTa and are newly initialized: ['pooler.dense

Epoch: [1][0/230] Elapsed 0m 0s (remain 1m 57s) Loss: 6.0719(6.0719) Grad: 61510.6133  LR: 0.00002000  
Epoch: [1][100/230] Elapsed 0m 9s (remain 0m 12s) Loss: 5.6578(5.8359) Grad: 81974.0859  LR: 0.00001991  
Epoch: [1][200/230] Elapsed 0m 19s (remain 0m 2s) Loss: 5.4834(5.7177) Grad: 136981.3438  LR: 0.00001963  
Epoch: [1][229/230] Elapsed 0m 22s (remain 0m 0s) Loss: 5.3924(5.6874) Grad: 105478.2344  LR: 0.00001951  
EVAL: [0/5] Elapsed 0m 0s (remain 0m 0s) Loss: 5.5552(5.5552) 


Epoch 1 - avg_train_loss: 5.6874  avg_val_loss: 5.4943  time: 23s
Epoch 1 - Score: 0.6767 0.4737
Epoch 1 - Save Best Score: 1.1504 Model


EVAL: [4/5] Elapsed 0m 0s (remain 0m 0s) Loss: 5.4313(5.4943) 
Epoch: [2][0/230] Elapsed 0m 0s (remain 0m 40s) Loss: 5.5051(5.5051) Grad: 76648.4609  LR: 0.00001951  
Epoch: [2][100/230] Elapsed 0m 9s (remain 0m 12s) Loss: 5.4131(5.3933) Grad: 104708.7188  LR: 0.00001900  
Epoch: [2][200/230] Elapsed 0m 18s (remain 0m 2s) Loss: 5.3361(5.3449) Grad: 317840.6875  LR: 0.00001833  
Epoch: [2][229/230] Elapsed 0m 21s (remain 0m 0s) Loss: 5.1277(5.3287) Grad: 108802.2422  LR: 0.00001810  
EVAL: [0/5] Elapsed 0m 0s (remain 0m 0s) Loss: 5.3489(5.3489) 


Epoch 2 - avg_train_loss: 5.3287  avg_val_loss: 5.3186  time: 22s
Epoch 2 - Score: 0.7594 0.5940
Epoch 2 - Save Best Score: 1.3534 Model


...

Epoch 9 - avg_train_loss: 4.4329  avg_val_loss: 4.7211  time: 23s
Epoch 9 - Score: 0.8045 0.6466


EVAL: [4/5] Elapsed 0m 0s (remain 0m 0s) Loss: 4.7681(4.7211) 
Epoch: [10][0/230] Elapsed 0m 0s (remain 0m 43s) Loss: 4.5139(4.5139) Grad: 46239.8750  LR: 0.00000052  
Epoch: [10][100/230] Elapsed 0m 9s (remain 0m 11s) Loss: 4.3133(4.4316) Grad: 48490.3711  LR: 0.00000017  
Epoch: [10][200/230] Elapsed 0m 18s (remain 0m 2s) Loss: 4.4323(4.4285) Grad: 47637.1016  LR: 0.00000001  
Epoch: [10][229/230] Elapsed 0m 21s (remain 0m 0s) Loss: 4.3848(4.4256) Grad: 54693.8125  LR: 0.00000000  
EVAL: [0/5] Elapsed 0m 0s (remain 0m 0s) Loss: 4.7408(4.7408) 


Epoch 10 - avg_train_loss: 4.4256  avg_val_loss: 4.7195  time: 22s
Epoch 10 - Score: 0.8045 0.6391


EVAL: [4/5] Elapsed 0m 0s (remain 0m 0s) Loss: 4.7692(4.7195) 


========== fold: 0 result ==========
Score: 0.8195 | 0.6466
========== CV ==========
Score: 0.8195 | 0.6466
